Phase 1: Problem Understanding & Dataset Preparation

In [1]:
# LOgistic regression and Random Forest
# model: Simple Neural network(dense layer), (RNN, CNN, LSTM, Bi-LSTM),  
# ML: decision tree, SVM, KNN, Naive Bayes, GBM(XGBoost, LightGBM, CatBoost), Extra Tree
# Transformer models: BERT, RoBerta, DistilBERT, ALBERT, mBERT

### Load the Required file

In [2]:
import pandas as pd

df=pd.read_csv("AI_Human.csv")
#df=df.head()    # limiting the dataset to few rows
df.head()

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0


### Pre-Processing the rows in the file

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Assume df is your DataFrame with 'text' and 'generated'

# Strip whitespace from text column
df['text'] = df['text'].str.strip()

# Encode labels first
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['generated'])

# Split first
X_train_text, X_test_text, y_train, y_test = train_test_split(
    df['text'], y, test_size=0.2, random_state=42, stratify=y
)

# Initialize TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

# Fit vectorizer on training data only
X_train = tfidf.fit_transform(X_train_text)

# Transform test data with the already fitted vectorizer
X_test = tfidf.transform(X_test_text)


### Logistic Regression Model

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc

# Train Logistic Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict class labels
y_pred = model.predict(X_test)

# Predict probabilities for ROC
y_prob = model.predict_proba(X_test)[:, 1]  # probability of class "1" (AI)

# Classification report
target_names = [str(c) for c in label_encoder.classes_]
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=target_names))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

# Compute ROC curve and AUC
fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_test, y_prob)
roc_auc_lr = auc(fpr_lr, tpr_lr)

# Store ROC variables in a dictionary for later use
roc_data_lr = {
    'fpr': fpr_lr,
    'tpr': tpr_lr,
    'thresholds': thresholds_lr,
    'roc_auc': roc_auc_lr,
}


Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99     61159
         1.0       0.99      0.98      0.99     36288

    accuracy                           0.99     97447
   macro avg       0.99      0.99      0.99     97447
weighted avg       0.99      0.99      0.99     97447

Accuracy: 0.9919


### Decision Tree Model


In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc

# Train Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Make predictions
dt_pred = dt_model.predict(X_test)
dt_prob = dt_model.predict_proba(X_test)[:, 1]  # probability of class "1" (AI)

# Evaluation
print("Decision Tree Classification Report:")
print(classification_report(y_test, dt_pred, target_names=['Human', 'AI']))
print(f"Accuracy: {accuracy_score(y_test, dt_pred):.4f}")

# Store ROC variables in a dictionary for later use
fpr_dt, tpr_dt, thresholds_dt = roc_curve(y_test, dt_prob)
roc_auc_dt = auc(fpr_dt, tpr_dt)

roc_data_dt = {
    'fpr': fpr_dt,
    'tpr': tpr_dt,
    'thresholds': thresholds_dt,
    'roc_auc': roc_auc_dt,
}

Decision Tree Classification Report:
              precision    recall  f1-score   support

       Human       0.99      0.99      0.99     61159
          AI       0.98      0.98      0.98     36288

    accuracy                           0.99     97447
   macro avg       0.98      0.98      0.98     97447
weighted avg       0.99      0.99      0.99     97447

Accuracy: 0.9851


### Random Forest Model

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
rf_pred = rf_model.predict(X_test)
rf_prob = rf_model.predict_proba(X_test)[:, 1]  # probability of class "1" (AI)

# Evaluation
print("Random Forest Classification Report:")
print(classification_report(y_test, rf_pred, target_names=['Human', 'AI']))
print(f"Accuracy: {accuracy_score(y_test, rf_pred):.4f}")

fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, rf_prob)
roc_auc_rf = auc(fpr_rf, tpr_rf)

roc_data_rf = {
    'fpr': fpr_rf,
    'tpr': tpr_rf,
    'thresholds': thresholds_rf,
    'roc_auc': roc_auc_rf,
}

Random Forest Classification Report:
              precision    recall  f1-score   support

       Human       1.00      1.00      1.00     61159
          AI       1.00      1.00      1.00     36288

    accuracy                           1.00     97447
   macro avg       1.00      1.00      1.00     97447
weighted avg       1.00      1.00      1.00     97447

Accuracy: 0.9982


### Simple Neural Network Model

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc
from tensorflow.keras.utils import Sequence
import numpy as np

# Custom generator to handle sparse matrix in batches
class TfidfBatchGenerator(Sequence):
    def __init__(self, X_sparse, y, batch_size):
        self.X_sparse = X_sparse
        self.y = y
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(self.X_sparse.shape[0] / self.batch_size))

    def __getitem__(self, idx):
        batch_X = self.X_sparse[idx * self.batch_size : (idx + 1) * self.batch_size].toarray().astype('float32')
        batch_y = self.y[idx * self.batch_size : (idx + 1) * self.batch_size]
        return batch_X, batch_y

# Create generators for training and validation
batch_size = 64
train_gen = TfidfBatchGenerator(X_train, y_train, batch_size)
test_gen = TfidfBatchGenerator(X_test, y_test, batch_size)

# Define and compile model
model_nn = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])
model_nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model_nn.fit(train_gen, validation_data=test_gen, epochs=5, verbose=1)

# Predict probabilities and labels
y_pred_prob = model_nn.predict(test_gen)
y_pred_nn = (y_pred_prob > 0.5).astype(int)

# Evaluate
print("Neural Network Classification Report:")
print(classification_report(y_test, y_pred_nn, target_names=['Human', 'AI']))
print(f"Accuracy: {accuracy_score(y_test, y_pred_nn):.4f}")

# Store ROC variables in dictionary for later use
fpr_nn, tpr_nn, thresholds_nn = roc_curve(y_test, y_pred_prob)
roc_auc_nn = auc(fpr_nn, tpr_nn)

roc_data_nn = {
    'fpr': fpr_nn,
    'tpr': tpr_nn,
    'thresholds': thresholds_nn,
    'roc_auc': roc_auc_nn,
}

c:\Python 3.10.11\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5


c:\Python 3.10.11\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6091/6091 ━━━━━━━━━━━━━━━━━━━━ 90s 14ms/step - accuracy: 0.9832 - loss: 0.0503 - val_accuracy: 0.9976 - val_loss: 0.0069
Epoch 2/5
6091/6091 ━━━━━━━━━━━━━━━━━━━━ 81s 13ms/step - accuracy: 0.9986 - loss: 0.0039 - val_accuracy: 0.9985 - val_loss: 0.0046
Epoch 3/5
6091/6091 ━━━━━━━━━━━━━━━━━━━━ 88s 14ms/step - accuracy: 0.9994 - loss: 0.0015 - val_accuracy: 0.9987 - val_loss: 0.0043
Epoch 4/5
6091/6091 ━━━━━━━━━━━━━━━━━━━━ 85s 14ms/step - accuracy: 0.9996 - loss: 0.0010 - val_accuracy: 0.9991 - val_loss: 0.0034
Epoch 5/5
6091/6091 ━━━━━━━━━━━━━━━━━━━━ 91s 15ms/step - accuracy: 0.9997 - loss: 6.2460e-04 - val_accuracy: 0.9990 - val_loss: 0.0038
1523/1523 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step
Neural Network Classification Report:
              precision    recall  f1-score   support

       Human       1.00      1.00      1.00     61159
          AI       1.00      1.00      1.00     36288

    accuracy                           1.00     97447
   macro avg       1.00      1.00      1.00     97

### RNN Model

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc
import numpy as np
from sklearn.model_selection import train_test_split

# Step 1: Tokenization and Padding
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
X_seq = pad_sequences(sequences, maxlen=max_len, padding='post')
y_seq = df['generated'].values

# Step 2: Train-test split
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# Step 3: Define RNN model
model_rnn = Sequential()
model_rnn.add(Embedding(input_dim=max_words, output_dim=64, input_length=max_len))
model_rnn.add(SimpleRNN(64, activation='tanh'))
model_rnn.add(Dense(1, activation='sigmoid'))

# Step 4: Compile model
model_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 5: Train model
model_rnn.fit(X_train_seq, y_train_seq, epochs=5, batch_size=32, validation_split=0.1, verbose=1)

# Step 6: Predict
y_pred_rnn_prob = model_rnn.predict(X_test_seq)
y_pred_rnn = (y_pred_rnn_prob > 0.5).astype(int)

# Step 7: Evaluation
print("RNN Classification Report:")
print(classification_report(y_test_seq, y_pred_rnn, target_names=['Human', 'AI']))
print(f"Accuracy: {accuracy_score(y_test_seq, y_pred_rnn):.4f}")

fpr_rnn, tpr_rnn, thresholds_rnn = roc_curve(y_test_seq, y_pred_rnn_prob)
roc_auc_rnn = auc(fpr_rnn, tpr_rnn)

roc_data_rnn = {
    'fpr': fpr_rnn,
    'tpr': tpr_rnn,
    'thresholds': thresholds_rnn,
    'roc_auc': roc_auc_rnn,
}

c:\Python 3.10.11\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
10963/10963 ━━━━━━━━━━━━━━━━━━━━ 642s 58ms/step - accuracy: 0.8363 - loss: 0.3770 - val_accuracy: 0.8689 - val_loss: 0.3166
Epoch 2/5
10963/10963 ━━━━━━━━━━━━━━━━━━━━ 556s 51ms/step - accuracy: 0.8525 - loss: 0.3308 - val_accuracy: 0.7808 - val_loss: 0.4529
Epoch 3/5
10963/10963 ━━━━━━━━━━━━━━━━━━━━ 480s 44ms/step - accuracy: 0.8723 - loss: 0.2936 - val_accuracy: 0.9228 - val_loss: 0.1977
Epoch 4/5
10963/10963 ━━━━━━━━━━━━━━━━━━━━ 473s 43ms/step - accuracy: 0.9001 - loss: 0.2432 - val_accuracy: 0.9114 - val_loss: 0.2252
Epoch 5/5
10963/10963 ━━━━━━━━━━━━━━━━━━━━ 476s 43ms/step - accuracy: 0.8565 - loss: 0.3151 - val_accuracy: 0.9409 - val_loss: 0.1801
3046/3046 ━━━━━━━━━━━━━━━━━━━━ 48s 16ms/step
RNN Classification Report:
              precision    recall  f1-score   support

       Human       0.93      0.97      0.95     61112
          AI       0.95      0.89      0.92     36335

    accuracy                           0.94     97447
   macro avg       0.94      0.93      0

### Bi-LSTM Model

In [10]:
# Import necessary libraries
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc
import numpy as np

# Step 1: Tokenization and Padding
max_words = 5000    # Maximum vocabulary size
max_len = 200       # Max sequence length for each input

# Initialize and fit tokenizer on the text data
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['text'])

# Convert texts to sequences of integers
sequences = tokenizer.texts_to_sequences(df['text'])

# Pad sequences so they are all the same length
X_seq = pad_sequences(sequences, maxlen=max_len, padding='post')
y_seq = df['generated'].values

# Step 2: Train-test split
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=42
)

# Step 3: Build the BiLSTM model
model_bilstm = Sequential()

# Embedding layer: transforms each word index into a 64-dimensional vector
model_bilstm.add(Embedding(input_dim=max_words, output_dim=64, input_length=max_len))

# Bidirectional LSTM layer
model_bilstm.add(Bidirectional(LSTM(64)))

# Output layer for binary classification
model_bilstm.add(Dense(1, activation='sigmoid'))

# Step 4: Compile the model
model_bilstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 5: Train the model
model_bilstm.fit(X_train_seq, y_train_seq, epochs=5, batch_size=32, validation_split=0.1, verbose=1)

# Step 6: Predict and evaluate
y_pred_bilstm_prob = model_bilstm.predict(X_test_seq).ravel()  # Flatten for ROC

# Convert probabilities to binary predictions
y_pred_bilstm = (y_pred_bilstm_prob > 0.5).astype(int)

# Print classification report
print("BiLSTM Classification Report:")
print(classification_report(y_test_seq, y_pred_bilstm, target_names=['Human', 'AI']))
print(f"Accuracy: {accuracy_score(y_test_seq, y_pred_bilstm):.4f}")

# Compute ROC curve variables for future use
fpr, tpr, thresholds = roc_curve(y_test_seq, y_pred_bilstm_prob)
roc_auc = auc(fpr, tpr)

# fpr, tpr, thresholds, roc_auc are ready for any future analysis or plotting
roc_data_blstm = {
    'fpr': fpr_rnn,
    'tpr': tpr_rnn,
    'thresholds': thresholds_rnn,
    'roc_auc': roc_auc_rnn,
}

c:\Python 3.10.11\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
10963/10963 ━━━━━━━━━━━━━━━━━━━━ 1573s 143ms/step - accuracy: 0.9476 - loss: 0.1387 - val_accuracy: 0.9899 - val_loss: 0.0305
Epoch 2/5
10963/10963 ━━━━━━━━━━━━━━━━━━━━ 1586s 145ms/step - accuracy: 0.9953 - loss: 0.0158 - val_accuracy: 0.9971 - val_loss: 0.0100
Epoch 3/5
10963/10963 ━━━━━━━━━━━━━━━━━━━━ 1738s 159ms/step - accuracy: 0.9983 - loss: 0.0055 - val_accuracy: 0.9983 - val_loss: 0.0063
Epoch 4/5
10963/10963 ━━━━━━━━━━━━━━━━━━━━ 1329s 121ms/step - accuracy: 0.9991 - loss: 0.0027 - val_accuracy: 0.9989 - val_loss: 0.0042
Epoch 5/5
10963/10963 ━━━━━━━━━━━━━━━━━━━━ 1500s 137ms/step - accuracy: 0.9994 - loss: 0.0018 - val_accuracy: 0.9987 - val_loss: 0.0054
3046/3046 ━━━━━━━━━━━━━━━━━━━━ 131s 43ms/step
BiLSTM Classification Report:
              precision    recall  f1-score   support

       Human       1.00      1.00      1.00     61112
          AI       1.00      1.00      1.00     36335

    accuracy                           1.00     97447
   macro avg       1.00   

### K-NN Model

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc

# Step 1: Train K-NN model
# Note: K-NN is sensitive to high-dimensional sparse data, so we use TF-IDF features
knn_model = KNeighborsClassifier(n_neighbors=5)  # You can tune this value
knn_model.fit(X_train, y_train)

# Step 2: Predict using the trained model
y_pred_knn = knn_model.predict(X_test)

# Step 3: Evaluate the model
print("K-NN Classification Report:")
print(classification_report(y_test, y_pred_knn, target_names=['Human', 'AI']))
print(f"Accuracy: {accuracy_score(y_test, y_pred_knn):.4f}")

# Step 4: Compute ROC curve variables for future use
# Get predicted probabilities for the positive class (AI=1)
y_pred_knn_prob = knn_model.predict_proba(X_test)[:, 1]

fpr_knn, tpr_knn, thresholds_knn = roc_curve(y_test, y_pred_knn_prob)
roc_auc_knn = auc(fpr_knn, tpr_knn)

# fpr_knn, tpr_knn, thresholds_knn, roc_auc_knn are ready for future analysis or plotting
roc_data_knn = {
    'fpr': fpr_rnn,
    'tpr': tpr_rnn,
    'thresholds': thresholds_rnn,
    'roc_auc': roc_auc_rnn,
}

K-NN Classification Report:
              precision    recall  f1-score   support

       Human       0.97      0.99      0.98     61159
          AI       0.98      0.95      0.96     36288

    accuracy                           0.97     97447
   macro avg       0.97      0.97      0.97     97447
weighted avg       0.97      0.97      0.97     97447

Accuracy: 0.9721


### SVM Model

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc

# Step 1: Initialize and train the SVM model
# 'linear' kernel is commonly used for text classification
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)

# Step 2: Make predictions
y_pred_svm = svm_model.predict(X_test)

# Step 3: Evaluate the model
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm, target_names=['Human', 'AI']))
print(f"Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}")

# Step 4: Compute ROC curve variables for future use
# Get predicted probabilities for the positive class (AI=1)
y_pred_svm_prob = svm_model.predict_proba(X_test)[:, 1]

fpr_svm, tpr_svm, thresholds_svm = roc_curve(y_test, y_pred_svm_prob)
roc_auc_svm = auc(fpr_svm, tpr_svm)

# fpr_svm, tpr_svm, thresholds_svm, roc_auc_svm are ready for future analysis or plotting
roc_data_svm = {
    'fpr': fpr_rnn,
    'tpr': tpr_rnn,
    'thresholds': thresholds_rnn,
    'roc_auc': roc_auc_rnn,
}

### XGBoost Model

In [12]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc

# Step 1: Initialize and train the XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

# Step 2: Predict on test data
y_pred_xgb = xgb_model.predict(X_test)

# Step 3: Evaluate performance
print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb, target_names=['Human', 'AI']))
print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")

# Step 4: Compute ROC curve variables for future use
# Get predicted probabilities for the positive class (AI=1)
y_pred_xgb_prob = xgb_model.predict_proba(X_test)[:, 1]

fpr_xgb, tpr_xgb, thresholds_xgb = roc_curve(y_test, y_pred_xgb_prob)
roc_auc_xgb = auc(fpr_xgb, tpr_xgb)

# fpr_xgb, tpr_xgb, thresholds_xgb, roc_auc_xgb are now ready for any future use
roc_data_xgboost = {
    'fpr': fpr_rnn,
    'tpr': tpr_rnn,
    'thresholds': thresholds_rnn,
    'roc_auc': roc_auc_rnn,
}


c:\Python 3.10.11\lib\site-packages\xgboost\training.py:183: UserWarning: [14:55:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost Classification Report:
              precision    recall  f1-score   support

       Human       0.99      1.00      1.00     61159
          AI       1.00      0.99      0.99     36288

    accuracy                           0.99     97447
   macro avg       0.99      0.99      0.99     97447
weighted avg       0.99      0.99      0.99     97447

Accuracy: 0.9937


### BERT Model

In [ ]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc
from scipy.special import softmax

# Assume df is already defined and has columns: 'text' and 'generated' (binary labels)
data_dict = {'text': df['text'].tolist(), 'label': df['generated'].tolist()}
dataset = Dataset.from_dict(data_dict)

# Split dataset
train_test = dataset.train_test_split(test_size=0.2, seed=42)
train_ds = train_test['train']
test_ds = train_test['test']

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenization function
def tokenize_fn(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=128)

train_ds = train_ds.map(tokenize_fn, batched=True)
test_ds = test_ds.map(tokenize_fn, batched=True)

train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Globals to hold ROC curve data after evaluation
roc_data = {}

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    # Convert logits to probabilities with softmax (for binary: 2 classes)
    probs = softmax(logits, axis=1)
    preds = np.argmax(probs, axis=1)
    
    # Print classification report and accuracy
    print(classification_report(labels, preds, target_names=['Human', 'AI']))
    acc = accuracy_score(labels, preds)
    print(f"Accuracy: {acc:.4f}")
    
    # Compute ROC curve and AUC for positive class (AI = 1)
    fpr, tpr, thresholds = roc_curve(labels, probs[:, 1])
    roc_auc = auc(fpr, tpr)
    
    # Store ROC variables globally for future use
    roc_data_bert = {
    'fpr': fpr,
    'tpr': tpr,
    'thresholds': thresholds,
    'roc_auc': roc_auc,
}
    
    return {"accuracy": acc}

# Training args
training_args = TrainingArguments(
    output_dir='./results_bert',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    evaluation_strategy='epoch',
    save_strategy='no',
    logging_dir='./logs_bert',
    logging_steps=10,
    seed=42,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

# Train model
trainer.train()

# Evaluate model (this triggers compute_metrics and fills roc_data)
trainer.evaluate()

# After evaluation, roc_data contains ROC variables and AUC, e.g.:
# print("ROC AUC:", roc_data['roc_auc'])


c:\Python 3.10.11\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Hemant Prakash\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For b

KeyboardInterrupt: 

### RoBERTa Model

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc
from scipy.special import softmax

# Prepare dataset (reuse df)
data_dict = {'text': df['text'].tolist(), 'label': df['label'].tolist()}
dataset = Dataset.from_dict(data_dict)
train_test = dataset.train_test_split(test_size=0.2, seed=42)
train_ds = train_test['train']
test_ds = train_test['test']

# Load RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Tokenize function
def tokenize_fn(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=128)

train_ds = train_ds.map(tokenize_fn, batched=True)
test_ds = test_ds.map(tokenize_fn, batched=True)

train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Convert logits to probabilities using softmax
    probs = softmax(logits, axis=1)
    preds = np.argmax(probs, axis=1)
    
    # Print classification report and accuracy
    print(classification_report(labels, preds, target_names=['Human', 'AI']))
    acc = accuracy_score(labels, preds)
    print(f"Accuracy: {acc:.4f}")
    
    # Compute ROC curve and AUC for positive class (AI=1)
    fpr, tpr, thresholds = roc_curve(labels, probs[:, 1])
    roc_auc = auc(fpr, tpr)
    
    roc_data['fpr'] = fpr
    roc_data['tpr'] = tpr
    roc_data['thresholds'] = thresholds
    roc_data['roc_auc'] = roc_auc
    roc_data_roberta = {
        'fpr': fpr_rnn,
         'tpr': tpr_rnn,
        'thresholds': thresholds_rnn,
        'roc_auc': roc_auc_rnn,
    }
    
    return {"accuracy": acc}

# Training arguments
training_args = TrainingArguments(
    output_dir='./results_roberta',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    evaluation_strategy='epoch',
    save_strategy='no',
    logging_dir='./logs_roberta',
    logging_steps=10,
    seed=42,
)

# Trainer initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model (fills roc_data)
trainer.evaluate()

# After evaluation, roc_data dict holds ROC variables:
# e.g., print("ROC AUC:", roc_data['roc_auc'])


ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

### Multilingual BERT(mBERT) Model

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc
from scipy.special import softmax

# Prepare dataset (reuse df)
data_dict = {'text': df['text'].tolist(), 'label': df['label'].tolist()}
dataset = Dataset.from_dict(data_dict)
train_test = dataset.train_test_split(test_size=0.2, seed=42)
train_ds = train_test['train']
test_ds = train_test['test']

# Load RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Tokenize function
def tokenize_fn(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=128)

train_ds = train_ds.map(tokenize_fn, batched=True)
test_ds = test_ds.map(tokenize_fn, batched=True)

train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    # Convert logits to probabilities using softmax
    probs = softmax(logits, axis=1)
    preds = np.argmax(probs, axis=1)
    
    # Print classification report and accuracy
    print(classification_report(labels, preds, target_names=['Human', 'AI']))
    acc = accuracy_score(labels, preds)
    print(f"Accuracy: {acc:.4f}")
    
    # Compute ROC curve and AUC for positive class (AI=1)
    fpr, tpr, thresholds = roc_curve(labels, probs[:, 1])
    roc_auc = auc(fpr, tpr)
    
    # Store ROC data globally for future use
    roc_data['fpr'] = fpr
    roc_data['tpr'] = tpr
    roc_data['thresholds'] = thresholds
    roc_data['roc_auc'] = roc_auc
    roc_data_mbert = {
        'fpr': fpr_rnn,
        'tpr': tpr_rnn,
        'thresholds': thresholds_rnn,
        'roc_auc': roc_auc_rnn,
    }
    
    return {"accuracy": acc}

# Training arguments
training_args = TrainingArguments(
    output_dir='./results_roberta',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    evaluation_strategy='epoch',
    save_strategy='no',
    logging_dir='./logs_roberta',
    logging_steps=10,
    seed=42,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

# Train
trainer.train()

# Evaluate (this triggers compute_metrics and fills roc_data)
trainer.evaluate()

# After evaluation, you can access ROC data like this:
# print("ROC AUC:", roc_data['roc_auc'])


ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

### Plotting ROC Curve for each model used

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))

# Logistic Regression (darkblue)
plt.plot(roc_data_lr['fpr'], roc_data_lr['tpr'],
         color='darkblue', lw=2,
         label=f'Logistic Regression (AUC = {roc_data_lr["roc_auc"]:.3f})')

# Decision Tree (darkgreen)
plt.plot(roc_data_dt['fpr'], roc_data_dt['tpr'],
         color='darkgreen', lw=2,
         label=f'Decision Tree (AUC = {roc_data_dt["roc_auc"]:.3f})')

# Random Forest (goldenrod)
plt.plot(roc_data_rf['fpr'], roc_data_rf['tpr'],
         color='goldenrod', lw=2,
         label=f'Random Forest (AUC = {roc_data_rf["roc_auc"]:.3f})')

# Simple Neural Network (crimson)
plt.plot(roc_data_nn['fpr'], roc_data_nn['tpr'],
         color='crimson', lw=2,
         label=f'Simple Neural Network (AUC = {roc_data_nn["roc_auc"]:.3f})')

# RNN (teal)
plt.plot(roc_data_rnn['fpr'], roc_data_rnn['tpr'],
         color='teal', lw=2,
         label=f'RNN (AUC = {roc_data_rnn["roc_auc"]:.3f})')

# BiLSTM (orangered)
plt.plot(roc_data_bilstm['fpr'], roc_data_bilstm['tpr'],
         color='orangered', lw=2,
         label=f'BiLSTM (AUC = {roc_data_bilstm["roc_auc"]:.3f})')

# K-NN (royalblue)
plt.plot(roc_data_knn['fpr'], roc_data_knn['tpr'],
         color='royalblue', lw=2,
         label=f'K-NN (AUC = {roc_data_knn["roc_auc"]:.3f})')

# SVM (forestgreen)
plt.plot(roc_data_svm['fpr'], roc_data_svm['tpr'],
         color='forestgreen', lw=2,
         label=f'SVM (AUC = {roc_data_svm["roc_auc"]:.3f})')

# XGBoost (purple)
plt.plot(roc_data_xgb['fpr'], roc_data_xgb['tpr'],
         color='purple', lw=2,
         label=f'XGBoost (AUC = {roc_data_xgb["roc_auc"]:.3f})')

# BERT (darkorange)
plt.plot(roc_data_bert['fpr'], roc_data_bert['tpr'],
         color='darkorange', lw=2,
         label=f'BERT (AUC = {roc_data_bert["roc_auc"]:.3f})')

# RoBERTa (saddlebrown)
plt.plot(roc_data_roberta['fpr'], roc_data_roberta['tpr'],
         color='saddlebrown', lw=2,
         label=f'RoBERTa (AUC = {roc_data_roberta["roc_auc"]:.3f})')

# Random baseline (black dashed)
plt.plot([0, 1], [0, 1], color='black', lw=1.5, linestyle='--',
         label='Random Guess (AUC = 0.5)')

# Labels and formatting
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('ROC Curve Comparison of All Models')
plt.legend(loc='lower right', fontsize='medium')
plt.grid(True)
plt.tight_layout()
plt.show()
